## Fleet and workforce planning with Linear Programming
#### A preview of Linear Programming's capabilities to solve some business problems
###### Author: Ezequiel Ortiz Recale

**Problem:** finding the optimal (less costly) way of transporting product boxes from warehouses to stores employing different vans and workers while fulfilling the constraints of demand, stock, vans and workers assignment.

**Context Variables**:
- "P" products = 5
- "W" warehouses = 3
- "S" stores = 3
- "V" vans = 4
- "E" employees = 8
- "J" conflicting pairs of employees = 23

**Assumptions**:
- We know the demand in boxes of each of our products, by store;
- We know how many boxes of our products are available in each of our warehouses;
- All the boxes have the same size (to simplify the problem);
- We know how many boxes fit in each of the vehicles;
- The cost of sending a box varies depending on the combination of product-warehouse-store-van;
- The use of each van has a fixed cost;
- The vans can't go from a warehouse to a store without transporting at least 1 box;
- The vans can't do more than 5 trips. We can assume that they all occur on the same day;
- There's a limit to how many boxes can be transported by each van;
- None of the vans can repeat the same trip;
- We need 2 employees assigned per van to be able to make a trip;
- If we assign an employee we have to pay him for this task 🙂;
- Each employee can only be assigned to a single van;
- It is desirable to avoid assigning conflicting employees to the same vehicle.

### Variables, Objective and Constraints

- $c_{pwsv}$: Unit cost of sending a box of product $p$ from warehouse $w$ to store $s$ using vehicle $v$, where $\space $   $c_{pwsv} \in (0,+\infty)$;

- $x_{pwsv}$: Number of boxes of product $p$ sent from warehouse $w$ to store $s$ using van $v$, where $\space  $ $x_{pwsv} \in [0,+\infty)$;

- $T_v$: Boolean variable created to verifiy that the vehicle $v$ is used, where $\space $ $T_v \in \{0,1\}$;

- $F_v$: Integer fixed cost for using the vehicle $v$, where $\space $ $F_v \in (0,+\infty)$;

- $A_{ve}$: Boolean variable created to verify if an employee $e$ is assigned to a van $v$, where $\space $ $A_{ve} \in \{0,1\}$;

- $Z_{wsv}$: Boolean auxiliary variable used to check if a trip from warehouse $w$ to store $s$ was made using van $v$ , where $\space $ $Z_{wsv} \in \{0,1\}$;

- $H_{vj}$: Boolean auxiliary variable to check if just one employee of a conflicting pair $j$ is assigned to a van $v$, where $\space $ $H_{vj} \in \{0,1\}$

- $G_{vj}$: Boolean variable to check if the conflicting pair $j$ is assigned to the same van $v$, where $\space $ $G_{vj} \in \{0,1\}$

$Min \sum\limits_{\substack{p\in P \\ w \in W \\ s \in S\\ v \in V}}c_{pwsv}x_{pwsv}+ \sum\limits_{\substack{v\in V}} T_{v}F_{v}+ \sum\limits_{\substack{v\in V\\ e \in E}} 1500A_{ve}+ \sum\limits_{\substack{v\in V\\ j \in J}}500G_{vj}$ $\ \$ (1)

$\ \ subject \ \ to$
 
- $\sum\limits_{\substack{w \in W \\ v \in V}} x_{pwsv} = demand_{ps}$ $\ \ \forall \ \ p \in P, \ \ s \in S$ $\ \$ (2)


- $\sum\limits_{\substack{s \in S \\ v \in V}}x_{pwsv} \le stock_{pw}$ $\ \ \forall \ \ p \in P, \ \ w \in W$ $\ \$ (3)


- $\sum\limits_{\substack{p\in P}} x_{pwsv} \le capacity_{v}Z_{wsv}$  $\ \ \forall \ \ w \in W, \ \ s \in S, \ \ v \in V$ $\ \$ (4)


- $\sum\limits_{\substack{w \in W \\ s \in S}}Z_{wsv} \le 5T_v$  $\ \ \forall \ \ v \in V$ $\ \$ (5)


- $\sum\limits_{\substack{e \in E}}A_{ve} = 2T_v$  $\ \ \forall \ \ v \in V$ $\ \$ (6)


- $\sum\limits_{\substack{v \in V}}A_{ve} \le 1$  $\ \ \forall \ \ e \in E$ $\ \$ (7)


- $\sum\limits_{\substack{v \in V}}A_{ve_1}+A_{ve_2}-H_{vj}-2G_{vj} = 0$ $\ \ \forall \ \ e \in E, \ \ j \in J \ \ $ 
with $ J =\{(e_a,e_b) \in E :(a,b) \ \ pairs \ \ with \ \ conflicts\}$ $\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \  \ \ \ \ \ \ \ \ \ \ \ \ \ \  \  \ \ and \ \ j = \{e_a \ne e_b : (e_a,e_b) \in J\}$ $\ \$ (8)


-  $x_{pwsv} \in [0,+\infty)$ $\  \  T_{v},\ \ Z_{wsv},\ \ A_{ve},\ \ H_{vj},\ \ G_{vj} \in \{0,1\}$ $\ \$ (9)


with $p=1,...,P, \space\space w=1,...,W,\space\space s=1,...,S \space\space v=1,...,V \space\space e=1,...,E \space\space j=1,...,J$

#### Imports

In [1]:
#!pip install ortools
import numpy as np
import pandas as pd
import itertools
from ortools.linear_solver import pywraplp

#### Full solution

In [2]:
#1) Specify a seed to be able to replicate the experiment
np.random.seed(2)

#2) Declare the context variables
W_warehouses = 3
S_stores = 3
P_products = 5
V_vans = 4
E_employees = 8
trip_limit = 5

#3) Set some thresholds for the simulated data
min_cost, max_cost = 10, 60
min_stock, max_stock = 8, 12
min_demand, max_demand = 1, 10
min_van_fixed_cost, max_van_fixed_cost = 3000, 5000
fixed_salary = 1500
conflict_penalty = 500

#4) GENERATE DATA
 # 1 Cost matrix per product (of size W_warehouses x S_stores x V_vans)
 # 1 Stock vector per product (of size W_warehouses)
 # 1 Demand vector per product (of size S_stores)
 # 1 Capacity list per van (i.e. how many boxes each can transport)
 # 1 List of pairs of conflicting employees

costs = []
stocks = []
demands = []
capacities = [5,6,6,4]
J_employees_conflicts = [(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),
                         (2,3),(2,4),(2,5),(2,6),(2,7),(2,8),
                         (3,4),(3,5),(3,6),(3,7),(3,8),
                         (4,5),(4,6),(4,7),(4,8),
                         (5,7)]

# For each product we'll generate random costs, stocks and demands, given by the intervals [low,high]
for i in range(P_products):
    costs_p=np.random.randint(low=min_cost,high=max_cost,size=W_warehouses*S_stores*V_vans).reshape(W_warehouses,S_stores,V_vans)
    stock_p=np.random.randint(low=min_stock,high=max_stock,size=W_warehouses)
    demand_p=np.random.randint(low=min_demand,high=max_demand,size=S_stores)

    costs.append(costs_p)
    stocks.append(stock_p)
    demands.append(demand_p)

# We create a vector of fixed costs for using each of the vans
for v in range(V_vans):
    costs_v=np.random.randint(low=min_van_fixed_cost,high=max_van_fixed_cost,size=V_vans)
    
    
# 5) CALL AN INSTANCE OF A SOLVER FOR MIXED INTEGER PROGRAMMING PROBLEMS
solver = pywraplp.Solver.CreateSolver('SCIP')

# 6) CREATE THE VARIABLES

# Variable X reflects the number of boxes shipped from a warehouse to a store using a specific van
x = {}
for p in range(P_products):
    for w in range(W_warehouses):
        for s in range(S_stores):
            for v in range(V_vans):
                # Write the variables following the previous mathematical notation x[Product,Warehouse,Store, Van]
                # Here we also specify that each quantity must be positive (from 0 to infinity)
                x[p,w,s,v] = solver.IntVar(lb=0,
                                           ub=solver.infinity(),
                                           name= f"x_{p+1}_{w+1}_{s+1}_{v+1}")
# Variable T reflects the use of van v
T = {}
for v in range(V_vans):
    T[v] = solver.BoolVar(name=f"T_{v+1}")

# Variable A signals if employee e is assigned to van v
A = {}
for v in range(V_vans):
    for e in range(E_employees):
        A[v,e] = solver.BoolVar(name=f"A_{v+1}_{e+1}")

# Variable Z is an auxiliary variable used to count the number of trips a van makes
Z = {}
for v in range(V_vans):
    for w in range(W_warehouses):
        for s in range(S_stores):
            Z[w,s,v] = solver.BoolVar(name=f"Z_{w+1}_{s+1}_{v+1}")

# Variable H signals that just 1 of member of the conflicting pair of employees j was assigned to van v
# Variable G signals that both members of a pair of conflicting employees j was a assigned to van v
H = {}
G = {}
for v in range(V_vans):
    for j in range(len(J_employees_conflicts)):
        H[v,j] = solver.BoolVar(name=f"H_{v+1}_{j+1}")
        G[v,j] = solver.BoolVar(name=f"G_{v+1}_{j+1}")
        
# 7) DEFINE THE CONSTRAINTS

# The sum of the columns (stock received at each store) must equal the store's demand
for p in range(P_products):
    for s in range(S_stores):
            solver.Add(solver.Sum([x[p, j[0], s, j[1]] for j in itertools.product(range(W_warehouses),range(V_vans))]) == demands[p][s],
                       name='2)Demand')

# The sum of stock of product p received at each store must be lower or equal to the available stock of p at each warehouse w
for p in range(P_products):
    for w in range(W_warehouses):
        solver.Add(solver.Sum([x[p, w, j[0],j[1]] for j in itertools.product(range(S_stores),range(V_vans))]) <= stocks[p][w],
                   name='3) Stock')


# None of the vans can make more than 5 trips
 # a) Register the possibility of making a trip from warehouse w to store s using van v
for v in range(V_vans):
    for s in range(S_stores):
        for w in range(W_warehouses):
            solver.Add(solver.Sum([x[p, w, s, v] for p in range(P_products)]) <= capacities[v]*Z[w, s, v],
                       name='4) TripVerification')

 # b) Limit the number of trips each van can make and check that a van was used
for v in range(V_vans):
    solver.Add(solver.Sum([Z[j[0], j[1],v] for j in itertools.product(range(W_warehouses),range(S_stores))]) <= trip_limit*T[v],
               name='5) TripLimit')

# The number of employees assigned to a van needs to 2 or 0
for v in range(V_vans):
    solver.Add(solver.Sum([A[v,e] for e in range(E_employees)]) == 2*T[v],
               name='6) EmployeeRequirement')

# An employee assigned to a van can't be assigned to another van
for e in range(E_employees):
    solver.Add(solver.Sum([A[v,e] for v in range(V_vans)]) <=1,name='7) JobLimit')

# Check if the desirable constraint is fulfilled
for v in range(V_vans):
    for idx,j in enumerate(J_employees_conflicts):
        solver.Add(solver.Sum([A[v,j[0]-1]])==-A[v,j[1]-1]+H[v,idx]+2*G[v,idx],
                   name='8) ConflictVerification')
        
# Define the objective function previously mentioned:
objective_function = []

# First term -> sum of variable costs of transportation
for p in range(P_products):
    for w in range(W_warehouses):
        for s in range(S_stores):
            for v in range(V_vans):
                objective_function.append(costs[p][w][s][v] * x[p, w, s, v])

# Second term -> sum of fixed costs of using a specific van
for v in range(V_vans):
    objective_function.append(costs_v[v]*T[v])

# Third term -> sum of salary payments per employee assigned to a task
for v in range(V_vans):
    for e in range(E_employees):
        objective_function.append(fixed_salary*A[v,e])

# Fourth term -> sum of penalties for not avoiding the joint assignment of conflicting employees
for v in range(V_vans):
    for j in range(len(J_employees_conflicts)):
        objective_function.append(conflict_penalty*G[v,j])

# Specify the type of problem. In this case, we want to minimize the objective function
solver.Minimize(solver.Sum(objective_function))

### Results

In [3]:
# Call the solver method to find the optimal solution
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print(f'\n Solution: \n Total cost = ${solver.Objective().Value()}')
else:
    print('A solution could not be found, check the problem specification')


 Solution: 
 Total cost = $22499.0


In [4]:
print(solver.ExportModelAsLpFormat(obfuscated=False))

\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 174
\   Variables        : 436
\     Binary         : 256
\     Integer        : 180
\     Continuous     : 0
Minimize
 Obj: +50 x_1_1_1_1 +25 x_1_1_1_2 +55 x_1_1_1_3 +18 x_1_1_1_4 +32 x_1_1_2_1 +53 x_1_1_2_2 +28 x_1_1_2_3 +21 x_1_1_2_4 +50 x_1_1_3_1 +17 x_1_1_3_2 +44 x_1_1_3_3 +59 x_1_1_3_4 +41 x_1_2_1_1 +21 x_1_2_1_2 +31 x_1_2_1_3 +57 x_1_2_1_4 +41 x_1_2_2_1 +36 x_1_2_2_2 +30 x_1_2_2_3 +47 x_1_2_2_4 +49 x_1_2_3_1 +13 x_1_2_3_2 +48 x_1_2_3_3 +14 x_1_2_3_4 +52 x_1_3_1_1 +53 x_1_3_1_2 +49 x_1_3_1_3 +48 x_1_3_1_4 +52 x_1_3_2_1 +43 x_1_3_2_2 +13 x_1_3_2_3 +15 x_1_3_2_4 +34 x_1_3_3_1 +14 x_1_3_3_2 +56 x_1_3_3_3 +16 x_1_3_3_4 +56 x_2_1_1_1 +22 x_2_1_1_2 +14 x_2_1_1_3 +36 x_2_1_1_4 +25 x_2_1_2_1 +59 x_2_1_2_2 +49 x_2_1_2_3 +56 x_2_1_2_4 +18 x_2_1_3_1 +55 x_2_1_3_2 +25 x_2_1_3_3 +51 x_2_1_3_4 +55 x_2_2_1_1 +18 x_2_2_1_2 +27 x_2_2_1_3 +32 x_2_2_1_4 +19 x_2_2_2_1 +51 x_2_2_2_2 +56 x_2_

#### Results extraction

In [5]:
result_list = []

for var in [x,Z,T,A,H,G]:
    variable_optimal = []
    for i in var.values():
        variable_optimal.append(i.solution_value())
    
    var_result=list(zip(var.values(),variable_optimal))
    
    df=pd.DataFrame(var_result,columns=['Name','Value'])
    
    result_list.append(df)

results=pd.concat(result_list)
results['Name']=results['Name'].astype(str)
results.reset_index(drop=True,inplace=True)
results['Variable']=results['Name'].str.extract("(^(.)\d?)")[0]
results['Variable']=results['Variable'].str.upper()
results['Value']=results['Value'].map(int)

variable_indices={'X':'X_product_warehouse_store_van',
                  'A':'A_van_employee',
                  'T':'T_van',
                  'H':'H_van_pair',
                  'G':'G_van_pair',
                  'Z':'Z_warehouse_store_van'}

results['Indices']=results['Variable'].map(variable_indices)

results=results[['Variable','Indices','Name','Value']].copy()

In [6]:
results.sample(5)

,Variable,Indices,Name,Value
152,X,X_product_warehouse_store_van,x_5_1_3_1,0
56,X,X_product_warehouse_store_van,x_2_2_3_1,0
161,X,X_product_warehouse_store_van,x_5_2_2_2,6
430,G,G_van_pair,G_4_18,0
80,X,X_product_warehouse_store_van,x_3_1_3_1,0


In [7]:
# Trucks to be used
list(results[(results['Variable']=='T')&(results['Value']==1)].Name)

['T_1', 'T_2', 'T_3']

In [8]:
# Trips of van 1

trips_df=results[(results['Variable']=='Z')&(results['Value']>0)]

trips_van_1=[]
for w in range(W_warehouses):
    for s in range(S_stores):
        for v in range(V_vans):
            if v==0:
                trips_van_1.append(str(Z[w,s,v]))


display(trips_df[trips_df['Name'].isin(trips_van_1)])

,Variable,Indices,Name,Value
180,Z,Z_warehouse_store_van,Z_1_1_1,1
182,Z,Z_warehouse_store_van,Z_1_3_1,1
184,Z,Z_warehouse_store_van,Z_2_2_1,1
187,Z,Z_warehouse_store_van,Z_3_2_1,1
188,Z,Z_warehouse_store_van,Z_3_3_1,1


In [9]:
# Boxes transported by van 1 during the trip from warehouse 2 to store 2
transport_df = results[(results['Variable']=='X')&(results['Value']>0)]

transport_trip_2_2 = []

for p in range(P_products):
    for w in range(W_warehouses):
        for s in range(S_stores):
            for v in range(V_vans):
                if w==1 and s==1 and v==0:
                    transport_trip_2_2.append(str(x[p,w,s,v]))

display(transport_df[transport_df['Name'].isin(transport_trip_2_2)])

,Variable,Indices,Name,Value
52,X,X_product_warehouse_store_van,x_2_2_2_1,4
88,X,X_product_warehouse_store_van,x_3_2_2_1,1


In [10]:
employees_van_1=[]
for v in range(V_vans):
    for e in range(E_employees):
        if v==0:
            employees_van_1.append(str(A[v,e]))
            
employees_df=results[(results['Variable']=='A')&(results['Value']>0)]

display(employees_df[employees_df['Name'].isin(employees_van_1)])

,Variable,Indices,Name,Value
224,A,A_van_employee,A_1_5,1
225,A,A_van_employee,A_1_6,1


In [11]:
# Conflicts
results[(results['Variable']=='G')&(results['Value']>0)]

,Variable,Indices,Name,Value
380,G,G_van_pair,G_2_14,1


In [12]:
# Conflicting employees assigned together to van 2
J_employees_conflicts[13]

(3, 4)

In [13]:
# We should be able to see that A_2_3 and A_2_4 both are equal to 1
results[(results['Variable']=='A')&(results['Value']!=0)]

,Variable,Indices,Name,Value
224,A,A_van_employee,A_1_5,1
225,A,A_van_employee,A_1_6,1
230,A,A_van_employee,A_2_3,1
231,A,A_van_employee,A_2_4,1
242,A,A_van_employee,A_3_7,1
243,A,A_van_employee,A_3_8,1
